In [1]:
from __future__ import unicode_literals, print_function

import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training import Example
import pickle

In [2]:
TRAIN_DATA = [
    ("Check the south facade of the building", {"entities": [(10, 15, "DIRECTION"), (30, 38, "BUILDING")]}),
    ("I wanna see how is the north side of the stadium look like", {"entities": [(23, 28, "DIRECTION"), (41, 48, "BUILDING")]}),
    ("Boss wanna see how is the west side of the hospital look like", {"entities": [(26, 30, "DIRECTION"), (43, 51, "BUILDING")]}),
    ("Go east side of that tower", {"entities": [(3, 7, "DIRECTION"), (21, 26, "BUILDING")]}),
    ("Fly above the playground", {"entities": [(4, 9, "DIRECTION"), (14, 24, "BUILDING")]}),
    ("Fly above the soccer field", {"entities": [(4, 9, "DIRECTION"), (14, 26, "BUILDING")]}),
    ("Fly to the west of the basketball court", {"entities": [(11, 15, "DIRECTION"), (23, 39, "BUILDING")]}),
    ("Go to the southwest side of the bridge", {"entities": [(10, 19, "DIRECTION"), (32, 38, "BUILDING")]}),
    ("Go to the southwest side of area B", {"entities": [(10, 19, "DIRECTION"), (28, 34, "BUILDING")]}),
    ("Check out the east of D", {"entities": [(14, 18, "DIRECTION"), (22, 23, "BUILDING")]}),
    ("Fly to the north of E", {"entities": [(11, 16, "DIRECTION"), (20, 21, "BUILDING")]}),
    ("Fly to the south of E and circle around", {"entities": [(11, 16, "DIRECTION"), (20, 21, "BUILDING"), (26, 39, "FUNCTION")]}),
    ("Check the northwest of Area A", {"entities": [(10, 19, "DIRECTION"), (23, 29, "BUILDING")]}),
    ("Take a screenshot on the northwest of Area A", {"entities": [(7, 17, "FUNCTION"), (25, 34, "DIRECTION"), (43, 44, "BUILDING")]}),
    ("Take a screenshot on the west of C", {"entities": [(7, 17, "FUNCTION"), (25, 29, "DIRECTION"), (33, 34, "BUILDING")]}),
    ("Go to the north of C and take a screenshot", {"entities": [(10, 15, "DIRECTION"), (19, 20, "BUILDING"), (32, 42, "FUNCTION")]}),
    ("circle around", {"entities": [(0, 13, "FUNCTION")]}),
    ("switch view", {"entities": [(0, 11, "FUNCTION")]}),
    ("switch view to FPV", {"entities": [(0, 11, "FUNCTION")]}),
    ("switch view to FPV", {"entities": [(0, 11, "FUNCTION")]}),
    ("fly", {"entities": [(0, 3, "FUNCTION")]}),
    ("continue fly", {"entities": [(9, 12, "FUNCTION")]}),
    ("flight", {"entities": [(0, 6, "FUNCTION")]}),
    ("pause", {"entities": [(0, 5, "FUNCTION")]}),
    ("pause flight", {"entities": [(0, 5, "FUNCTION")]}),
    ("stop BPC", {"entities": [(0, 8, "FUNCTION")]}),
    ("stop bpc", {"entities": [(0, 8, "FUNCTION")]}),
    ("start BPC", {"entities": [(0, 9, "FUNCTION")]}),
    ("start bpc", {"entities": [(0, 9, "FUNCTION")]}),
    ("take off", {"entities": [(0, 8, "FUNCTION")]}),
]

In [3]:
model = None
output_dir=Path("ner/")
n_iter=100

   #load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [4]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

In [5]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

100%|██████████| 28/28 [00:00<00:00, 50.54it/s]


{'ner': 105.79564296081662}


100%|██████████| 28/28 [00:00<00:00, 51.58it/s]


{'ner': 72.7896251231432}


100%|██████████| 28/28 [00:00<00:00, 51.18it/s]


...

{'ner': 4.104745218018226}


100%|██████████| 28/28 [00:00<00:00, 48.07it/s]


{'ner': 3.3142589530114366}


100%|██████████| 28/28 [00:00<00:00, 49.12it/s]

{'ner': 3.169738259078391}


In [6]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "aecNlp.pkl", "wb" ))

Saved model to ner


In [7]:
doc=nlp("Go to the northwest of the fountain and take a screenshot")
print(doc.ents)

(northwest, fountain, screenshot)


In [8]:
doc=nlp("professors said they wanna check the northeast windows of the apartment")
print(doc.ents)

(professors, northeast, apartment)


In [9]:
doc=nlp("Check out southwest of C and screenshot")
print(doc.ents)

(southwest, C, screenshot)


In [10]:
doc=nlp("Fly to southwest of C and circle around")
print(doc.ents)

(southwest, C, circle around)


In [12]:
doc=nlp("switch view to fpv")
print(doc.ents)

(switch view,)
